In [7]:
import itertools as itools
import numpy as np

# For this challenge some simple modifications at the IntcodeComputer function (opcode 3 and 4)
# are sufficient. The rest is implemented outside the function. Not like Part 2... (ouch)

# Documentation for various intcodes
def documentation():
    
    print("One or more user input are needed to run this Intcode.")
    print("Please run as: 'IntcodeComputer(Intcode, [USER_INPUT_1, USER_INPUT_2, ...])'")
    print("The software can also be run in 'feedback' mode, upon provising the explicit argument")
    print("in the function call as 'IntcodeComputer(Intcode, user_input, feedback = True).'")
    print("where USER_INPUT_i is an integer number. Refer to the text of the exercise")
    print("for how to run this computer.")
    #print(" - 1 for diagnostics")
    #print(" - 5 for Thermal Radiator Controller ")
    
    return 0


# A structured data type for instructions
class Instruction:
        
    def __init__(self, code):
        
        # Sets that associates an opcode with its associated number of parameters
        # 3 parameters opcodes
        III_params = [1, 2, 7, 8]
        # 2 parameters opcodes
        II_params = [5, 6]
        # 1 parameter opcodes
        I_params = [3, 4]
        
        # Variables of the instruction set
        # Increment value after instruction
        self.increment = 0
        # Number of parameters
        self.n_params = 0
        # Parameter modes
        self.param_mode = []
        # Control variable
        self.is_corrupted = False
        # Number of digits that make up the opcode
        opcode_digits = 2
        
        lscode = list(str(code))
        self.opcode = int(''.join(lscode[-opcode_digits:])) # This works even if len(o) = 1!
        
        if self.opcode in III_params:
            self.increment = 4
            self.n_params = 3
            # My favorite line <3
            #self.param_mode = [int(x) if len(o) > 2 else 0 for x in reversed(o[:-2])]
        elif self.opcode in I_params:
            self.increment = 2
            self.n_params = 1
            
        elif self.opcode in II_params:
            self.increment = 3
            self.n_params = 2
       
        elif self.opcode == 99:
            pass
            
        else:
            self.is_corrupted = True
                    
        for i in range(self.n_params):
            try:
                rlscode = lscode[::-1]    # Reversed 'lscode'
                self.param_mode.append(int(rlscode[opcode_digits + i]))
                
            except:
                self.param_mode.append(0)
         
    def ShowInstructionData(self, code):
        print("Instruction: ", code)
        print("Opcode: ", self.opcode)
        print("Increment: ", self.increment)
        print("Number of parameters: ", self.n_params)
        print("parameters mode: ", self.param_mode)
        
                
# The Intcode computer
def IntcodeComputer(sequence, user_input = None):
    
    # New global variable, see later (opcode 3)
    glob_input_count = 0
    
    curr_pos = 0
    i = Instruction(sequence[curr_pos])
    
    while(i.opcode != 99): 
        
        #i.ShowInstructionData(sequence[curr_pos])
        if not i.is_corrupted:
            
            # Initialize variables if needed for execution of the instruction
            if i.n_params > 0:
                a = sequence[curr_pos+1] if i.param_mode[0] else sequence[sequence[curr_pos+1]]
            if i.n_params >= 2:
                b = sequence[curr_pos+2] if i.param_mode[1] else sequence[sequence[curr_pos+2]]
   
            # Add
            if i.opcode == 1:
                if i.param_mode[2]:
                    sequence[curr_pos+3] = a+b
                else:
                    sequence[sequence[curr_pos+3]] = a+b
                    
                curr_pos += i.increment
            
            # Multiply
            elif i.opcode == 2:
                if i.param_mode[2]:
                    sequence[curr_pos+3] = a*b
                else:
                    sequence[sequence[curr_pos+3]] = a*b
                    
                curr_pos += i.increment
                
            # Write input to address    
            elif i.opcode == 3:
                
                if user_input is not None :
                    # We have a long input (more than just 1 number)
                    # We access the user input one item at a time.
                    sequence[sequence[curr_pos+1]] = user_input[glob_input_count]
                    glob_input_count += 1
                    
                else:
                    documentation()
                    return 0
                    
                curr_pos += i.increment
                
            
            # Print output from address
            elif i.opcode == 4:
                output = sequence[curr_pos+1] if i.param_mode[0] else sequence[sequence[curr_pos+1]]
                
                #print("Output: ", output, '\n')
                curr_pos += i.increment
             
            # jump-if-True
            elif i.opcode == 5:
                if a != 0:
                    curr_pos = b
                else:
                    curr_pos += i.increment
            
            # jump-if-false
            elif i.opcode == 6:
                if a == 0:
                    curr_pos = b
                else:
                    curr_pos += i.increment
            
            # less-than
            elif i.opcode == 7:
                if i.param_mode[2]:
                    sequence[curr_pos+3] = 1 if a < b else 0
                else:
                    sequence[sequence[curr_pos+3]] = 1 if a < b else 0
                    
                curr_pos += i.increment
            
            # equal
            elif i.opcode == 8:
                if i.param_mode[2]:
                    sequence[curr_pos+3] = 1 if a == b else 0
                else:
                    sequence[sequence[curr_pos+3]] = 1 if a == b else 0
                    
                curr_pos += i.increment
        
        # If opcode is unknown
        else:
            print("Unknown opcode '" + str(i.opcode) + "'. Terminating.")
            return -1
   
        # Read the future instruction
        i = Instruction(sequence[curr_pos])
        
     
    #print("Program is finished. Haulting.")
    
    # We return the output variable from opcode 4. This can be done here or inside
    # the opcode 4 option, since the program halts immediately after the output.
    return output


In [8]:
# Load input data from file
with open("input.txt", 'r') as infile:
    strIntcode = infile.read().split(',')
    
Intcode = np.array([int(x) for x in strIntcode])

In [9]:
max_output = 0
phase_seq = []
# Cycle over permutations of [0,1,2,3,4]
for it in itools.permutations(range(5)):
    
    # Reset input signal
    input_signal = 0
    output_signal = 0
    
    for phase in it:
        # Reset Amplifier Controller Software (ACS)
        Intcode = np.array([int(x) for x in strIntcode])

        # Initialize input sequence for Software
        u_input = list([phase, input_signal])        
     
        # Run ACS on amplifier and save output
        output_signal = IntcodeComputer(Intcode, u_input)
        
        # Set connect the output to new input
        input_signal = output_signal
    
    # Look for max value
    if output_signal > max_output:
        max_output = output_signal
        phase_seq = it
                

print(max_output, phase_seq)

338603 (3, 0, 2, 1, 4)
